In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import input_data

In [4]:
mnist = input_data.read_data_sets('data/',one_hot=True)
trainimg = mnist.train.images
trainlabel = mnist.train.labels
testimg = mnist.test.images
testlabel = mnist.test.labels
print('MNIst loaded')

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
MNIst loaded


In [7]:
print(trainimg.shape)
print(trainlabel.shape)
print(testimg.shape)
print(testlabel.shape)

(55000, 784)
(55000, 10)
(10000, 784)
(10000, 10)


In [11]:
print(trainimg)
print(trainlabel[0]) # 做10分类   是哪一类就是1

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


### 开始实现逻辑回归

In [49]:
x = tf.placeholder(dtype=tf.float32,shape=[None,784])
y = tf.placeholder(dtype=tf.float32,shape=[None,10]) # 我们不知道有多少行
# W = tf.Variable(tf.random_normal([784,10],mean=0,stddev=1))
# b = tf.Variable(tf.random_normal([10],mean=0,stddev=1))
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
# 使用激活函数
actv = tf.nn.sigmoid(tf.matmul(x,W)+b)
# 定义损失函数
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(actv),reduction_indices=1))
# optimizer
learning_rate = 0.01
optm = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
# 模型搭建完成

In [50]:
# prediction 比较预测值的索引和真实值的索引
pred = tf.equal(tf.argmax(actv,1),tf.argmax(y,1))
# 求出精度
accr = tf.reduce_mean(tf.cast(pred,dtype=tf.float32)) # cast 就是转化类型
# 初始化
init= tf.global_variables_initializer()

In [51]:
arr = np.array([[31, 23,  4, 24, 27, 34],
                 [18,  3, 25,  0,  6, 35],
                 [28, 14, 33, 22, 20,  8],
                 [13, 30, 21, 19,  7,  9],
                 [16,  1, 26, 32,  2, 29],
                 [17, 12,  5, 11, 10, 15]])
with tf.Session() as sess:
    print(tf.rank(arr).eval()) #  矩阵的维度
    print(tf.shape(arr).eval()) # 行数和列数
    print(tf.arg_max(arr,0).eval()) # 0 是指列   在某一列最大值的索引
    print(tf.arg_max(arr,1).eval())

2
[6 6]
[0 3 2 4 0 1]
[5 5 2 1 3 0]


In [52]:
train_epochs = 50 # 所有的样本迭代50次
batch_size = 100 # 每迭代一次要100个样本
display_step = 5
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(train_epochs):
        avg_cost = 0  # 损失值
        num_batch = int(mnist.train.num_examples/batch_size) # 迭代的次数
        for i in range(num_batch):
            batch_xs,batch_ys  = mnist.train.next_batch(batch_size)
            # 进行一次优化
            sess.run(optm,feed_dict={x:batch_xs,y:batch_ys})
            # 求出损失值
            avg_cost +=sess.run(cost,feed_dict={x:batch_xs,y:batch_ys})/num_batch
        
        #DISPLAY
        if epoch% display_step==0:
            feeds_train = {x:batch_xs,y:batch_ys}
            feed_test = {x:mnist.test.images,y:mnist.test.labels}
            train_acc = sess.run(accr,feed_dict=feeds_train)
            test_acc = sess.run(accr,feed_dict=feed_test)
            print('Epoch:%03d/%03d cost:%.9f train_acc:%.3f test_acc:%.3f'
                 %(epoch,train_epochs,avg_cost,train_acc,test_acc))
print('DONE')

Epoch:000/050 cost:0.153199633 train_acc:0.780 test_acc:0.795
Epoch:005/050 cost:0.011759029 train_acc:0.680 test_acc:0.737
Epoch:010/050 cost:0.006563834 train_acc:0.660 test_acc:0.718
Epoch:015/050 cost:0.004627980 train_acc:0.760 test_acc:0.705
Epoch:020/050 cost:0.003602935 train_acc:0.610 test_acc:0.698
Epoch:025/050 cost:0.002963874 train_acc:0.620 test_acc:0.694
Epoch:030/050 cost:0.002525450 train_acc:0.650 test_acc:0.689
Epoch:035/050 cost:0.002205039 train_acc:0.680 test_acc:0.686
Epoch:040/050 cost:0.001960118 train_acc:0.670 test_acc:0.683
Epoch:045/050 cost:0.001766501 train_acc:0.660 test_acc:0.681
DONE
